In [ ]:
import numpy as np
from numpy.random import multivariate_normal, normal
from numpy import linalg

In [1]:
#initialise parameters
N = 100 # Number of iterations
M = 5 # dimensionality of state
e = 1e-5 # convergence condition

R = [] # Process noise Covariance
Q = [] # Measurement Noise
z = [] # initial state
A = [] # state transition
B = [] # noise matrix
C = [] # observation matrix

y = []
E = linalg.inv(np.block([[Q,np.zeros(M,M)],[np.zeros([M,M]),R]]))

In [ ]:
def pdmm_kalman():
    z = normal(0,1,[M,1])
    u = normal(0,1,[M,1])
    error = 1
    while error > e:
        ml_cost(z,u)
        z_new = z +  
        u_new = u + 
    return 

In [ ]:
def ml_cost(z,u,y):
    x = np.block([u,y-np.dot(C,z)])
    return np.dot(np.dot(x,E),x.T)

In [ ]:
for i in range(N):
    u = multivariate_normal(np.zeros(M),Q).T
    v = multivariate_normal(np.zeros(M),R).T
    z = np.dot(A,z) + np.dot(B,u)
    y = np.dot(C,u) + v
    
    